# 2012 - 2016 Median Household Income by State Heatmap

Read in 2012, 2013, 2014, 2015 and 2016 Median Household Income from census.gov, and create a heat map by US State.

In [1]:
# Dependencies:
# First you need to pip install plotly and create free account online for 4the api key. 
# Second you need to pip install Orca.
# Then you can call plotly as py and orca as pio.
import pandas as pd
import numpy as np
import requests
import plotly
plotly.tools.set_credentials_file(username='drbeaman', api_key='sT7AGDihwOZJVDF5iwhl')
import plotly.plotly as py
import plotly.io as pio

In [2]:
# Import incidents by state file as DataFrame
median_inc = "Resources/median_household_income.csv"
median_inc_df = pd.read_csv(median_inc)

In [3]:
# Preview df
median_inc_df.head()

,State code,State,Income,Margin Of Error
0,AL,Alabama,44758,+/- $314
1,AK,Alaska,74444,+/- $809
2,AZ,Arizona,51340,+/- $231
3,AR,Arkansas,42336,+/- $234
4,CA,California,63783,+/- $188


In [4]:
# Check the data types to see exact names of columns to rename
median_inc_df.dtypes

State code         object
State              object
Income              int64
Margin Of Error    object
dtype: object

In [5]:
# Drop unnecessary columns
median_inc_df = median_inc_df.drop(columns=['Margin Of Error'])
median_inc_df

,State code,State,Income
0,AL,Alabama,44758
1,AK,Alaska,74444
2,AZ,Arizona,51340
3,AR,Arkansas,42336
4,CA,California,63783
5,CO,Colorado,62520
6,CT,Connecticut,71755
7,DE,Delaware,61017
8,FL,Florida,48900
9,GA,Georgia,51037


In [6]:
#Rename columns
median_inc_df.rename(columns=
    {'Income':'Median Household Income'}, inplace=True)
median_inc_df.columns

Index(['State code', 'State', 'Median Household Income'], dtype='object')

In [7]:
# Preview DataFrame
median_inc_df.head()

,State code,State,Median Household Income
0,AL,Alabama,44758
1,AK,Alaska,74444
2,AZ,Arizona,51340
3,AR,Arkansas,42336
4,CA,California,63783


In [8]:
median_inc_df.sort_values(by='Median Household Income', ascending=True)

,State code,State,Median Household Income
23,MS,Mississippi,40528
3,AR,Arkansas,42336
47,WV,West Virginia,42644
0,AL,Alabama,44758
16,KY,Kentucky,44811
17,LA,Louisiana,45652
30,NM,New Mexico,45674
41,TN,Tennessee,46574
39,SC,South Carolina,46898
35,OK,Oklahoma,48038


# Map data

In [18]:
#For loop
for col in median_inc_df.columns:
    median_inc_df[col] = median_inc_df[col].astype(str)

#Set color groups for heat scale 
'''scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
    [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]
'''
# Prepare data to put into text.
# Text variable will be inserted into 'data' dictionary below.
# These columns will appear in plotly.com hover feature.
median_inc_df['text'] = median_inc_df['State'] + '<br>' +\
    ' Median Household Income ' + median_inc_df['Median Household Income']

# Set up data dictionary to hold the data
data = [ dict(
        type='choropleth',
        colorscale = 'Hot',
        autocolorscale = False,
        reversescale = False,
        locations = median_inc_df['State code'],
        z = median_inc_df['Median Household Income'].astype(float),
        locationmode = 'USA-states',
        text = median_inc_df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = "Income"
        )
    ) ]

# Set up layout dictionary for the visualization
layout = dict(
        title = '2012-2016 Median Household Income by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)',
        ),
    )

# Name the figure as a dictionary of the 1) data dictionary and 2) layout dictonary 
fig = dict( data=data, layout=layout )


#save fig
pio.write_image(fig, 'Images/1216_median_income.png')

# Name the file for your inline viewing 
# url = py.iplot( fig, filename='d3-cloropleth-map' )
py.iplot(data, filename='d3-cloropleth-map-income')